# Partial Differential Equations with Dedalus

Start with the heat equation in one dimension,
$$
\partial_t T = \kappa \partial_x^2 T.
$$
In one sense, this is a very pointless equation to solve numerically, since it is a linear, constant coefficient PDE that can be solved exactly. However, we begin with it for two reasons: we have a good *physical* intuition of what the solution will do, and we can make a *quantitative* comparison between our numerical method and the exact solution. I assure you that this is not just a pedegogical exercise that you won't do later in your career. Testing against (and constructing) known analytic solutions (for the mathematically minded student: this is just physics-speak for an exact solution, usually in closed form.) is an **essential part of numerical work**. It is akin to calibrating laboratory instruments. 


Of course, any PDE requires both initial and boundary conditions. To understand the number of boundary conditions is somewhat complicated, but for the kind of approximations that we are making to solve them, there needs to be **one per spatial derivative**, so in this case, we need two. 


## First Order Form
`d3` can solve differential equations of arbitrary spatial order without performance loss, but for all of the examples we will consider, we will reduce all of the equations to an equivalent first order form. This should be familiar to you from Newton's second law, which we can write as a system of first order equations for $x$ and $v$:

$$
\partial_t \begin{bmatrix} x\\ v\end{bmatrix} = \begin{bmatrix} v\\ F \end{bmatrix}.
$$

For the heat equation, we can write $T_x = \partial_x T$, and then we have the system
$$
\begin{align}
\partial_t T &= \kappa \partial_x T_x\\
T_x &= \partial_x T.
\end{align}
$$

Remember what we are ultimately trying to do: find a **polynomial** solution to our problem. That is, we are projecting all of our fields against some finite set of polynomials. 

Dedalus uses the $\tau$ method to do this, which is essentially to *modify the equation* such that a polynomial is an exact solution to the modified equation. For us, that will look like

$$
\partial_t T - \kappa \partial_x^2 T + \tau_1 P_1(x) + \tau_2 P_2(x) = 0.
$$

The $\tau$ variables are scalar variables in this case, and $P_n(x)$ are some polynomials, which we are free to choose.

If we have discretized $T$ on a set of $N$ polynomials, then we have $N$ degrees of freedom,

$$
T(x) = \hat{T}_0 P_0(x) + \hat{T}_1 P_1(x) + \cdots + \hat{T}_{N-1} P_{N-1}(x).
$$

The $\tau$ method allows us to impose boundary conditions: we now have two additional degrees of freedom, $\tau_1$ and $\tau_1$ that can 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dedalus.public as d3
from load_d3 import load_d3
import os

In [ ]:
nx = 1024
Lx = 1

# Bases
xcoord = d3.Coordinate('x')
dist = d3.Distributor(xcoord, dtype=np.float64)
xbasis = d3.Chebyshev(xcoord, nx, bounds=(0, Lx), dealias=1)

# Fields
T = dist.Field(name='T', bases=xbasis)
tau1 = dist.Field(name='tau1')
tau2 = dist.Field(name='tau2')

# Problem
problem = d3.IVP([T, tau1, tau2], namespace=locals())

In [ ]:
# Substitutions
dx = lambda A: d3.Differentiate(A, xcoord)
kappa = 1

# Tau polynomials
tau_basis = xbasis.derivative_basis(1)
p1 = dist.Field(bases=tau_basis)
p2 = dist.Field(bases=tau_basis)
p1['c'][-1] = 1
p2['c'][-1] = 1

Tx = dx(T) + p2*tau2

In [ ]:
# Add main equation, with linear terms on the LHS and nonlinear terms on the RHS
problem.add_equation("dt(T) - kappa*dx(Tx)  + p1*tau1 = 0")

# Add boundary conditions
problem.add_equation("T(x='left') = 0")
problem.add_equation("T(x='right') = 0")

Dedalus rule of thumb: if you have no other knowledge, use `RK222` as your timestepper.

In [ ]:
# Build solver
solver = problem.build_solver(d3.RK222)

When to stop? 

**Exercise:** Use dimensional analysis to find a *characteristic time scale* for this problem.

In [ ]:
# Stopping criteria
solver.stop_sim_time = 0.5

In [ ]:
x0 = 0.5
sigma = 0.05

In [ ]:
# Setup a gaussian pulse
x = dist.local_grid(xbasis)
T['g'] = np.exp(-(x - x0)**2/sigma**2)

In [ ]:
if not os.path.exists('heat'):
    os.mkdir('heat')
check = solver.evaluator.add_file_handler('heat/checkpoints', iter=10, parallel='gather')
check.add_tasks(solver.state)

In [ ]:
# Main loop
timestep = 0.001
while solver.proceed:
    solver.step(timestep)
    if solver.iteration % 200 == 0:
        print('Completed iteration {}'.format(solver.iteration))

In [ ]:
df = load_d3("heat/checkpoints/checkpoints_s1.h5")

A very good way to visualize 1-D problems is what is called a *space-time diagram* (in climate science, this is frequently called a *Hovmöller diagram*). This is a simply a "heat map" of values on a time-space grid. We use `plt.pcolormesh` to do this because of the non-uniform grid of Chebyshev polynomials.

In [ ]:
# Plot solution
plt.figure(figsize=(10,10))
df['T'].plot(x="x", y="t")

Note that this solution decays so fast that the evolution is hardly visible in this plot. Let's plot several times on a 1D plot:

In [ ]:
df['T'][:10].plot(x="x", col="t", col_wrap=3)

## Comparing against analytical solution

The analytical solution to this problem is 

$$
T(t,x) = \sum_k B_k \exp(-\kappa k^2 t),
$$

where $B_n$ are the the Fourier coefficients of the initial conditions,

$$
B_k = \int T(0,x) \sin(n\pi x/L_x).
$$

Note that this analytical solution is defined on Fourier, not Chebyshev polynomials, as is typical for linear, constant coefficient, homogeneous PDEs.

**Exercise:** Create a fourier basis and compute $B_N$, then use that to construct the analytic solution and check your Dedalus solution against it.